In [3]:
import os
import json
import math
import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize
import matplotlib
import matplotlib.pyplot as plt
import cv2
import scipy.misc

from skimage.draw import line_aa
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon, MultiPolygon
from shapely.ops import cascaded_union
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import imageio

# Création d'un masque à partir du GeoJson et d'une liste de features

- Cette version traite une image à la fois. 
**V2 possible: prendre en entrée un fichier texte contenant les noms des cartes à traiter**
- On considère une liste donnée de feattypes donnée plus bas
- Le masque en sortie est un png de la même taille que le GeoTIFF en entrée, codé sur un canal 8 bits
- Si un pixel appartient à au moins deux objets, il prend la valeur de pixel somme 
**V2 possible: donner une règle de priorité entre les types d'objet: qui passe devant qui ? ex runway marking devant runwayelement.**


In [46]:
# A MODIFIER AVEC LES BONS CHEMINS POUR RUN LE CODE SUR VOTRE MACHINE

path_GeoTIFF = "/home/odemoly/Documents/Supaero 4A/PIE/Data/GeoTIFF/"
path_GeoJson = "/home/odemoly/Documents/Supaero 4A/PIE/Data/GeoJson/"

#image considérée
name_map = "LFBO"
files=[]
for i in os.listdir(path_GeoJson):
    if os.path.isfile(os.path.join(path_GeoJson,i)) and name_map in i:
        files.append(i)
        
filepath = path_GeoJson+files[0]

#chemin de sortie pour le masque png
outfolder = "/home/odemoly/Documents/Supaero 4A/PIE/Masks"  
os.makedirs(outfolder, exist_ok=True)

In [53]:
# Liste des features d'intérêt et color mapping (1 canal nuances de gris de 1 à 255, 0 est background)
color_mapping = {
    'Apron.apronelement': 10,
    'Runway.runwayelement': 50,
    'ServiceRoad.serviceroad': 100,
    'Runway.runwaymarking':120,
    'Runway.runwaydisplacedarea':140,
    'Runway.runwayshoulder':160,
    'Taxiway.taxiwayshoulder':180,
    'ParkingStandArea.parkingstandarea':200,
    'FinalApproachAndTakeOffArea.finalapproachandtakeoffarea':220,
    'TouchDownLiftOffArea.touchdownliftoffarea':240,
    'Runway.runwayintersection':50,
    #0 correspond à Taxiway.taxiwayelement
    '0':70,
    'Taxiway.taxiwayholdingposition':255,
    'LandAndHoldShortOperationLocation.landandholdshortoperationlocation':252,
    'Runway.runwayexitline':248
}

#List of features to identify.
#The order in this list matters as if there is superposition between 2 features or more,
#the last coming in the list will dominates the others and give its color to the corresponding pixels. 
features_list = ['0','Apron.apronelement','Runway.runwayelement','Runway.runwayintersection','Runway.runwaydisplacedarea','Runway.runwayshoulder','Runway.runwaymarking','ServiceRoad.serviceroad','Taxiway.taxiwayshoulder',
                 'ParkingStandArea.parkingstandarea','FinalApproachAndTakeOffArea.finalapproachandtakeoffarea','TouchDownLiftOffArea.touchdownliftoffarea']
line_list = ['Taxiway.taxiwayholdingposition','LandAndHoldShortOperationLocation.landandholdshortoperationlocation','Runway.runwayexitline']


In [97]:
def draw_linestring(line_list,whole_mask,transform):
    #transform = src.meta['transform']
    df = func_df(filepath)
    df = df[df['geometry'].geom_type=='LineString']
    for k in df.id.keys():
        for ls in line_list:
            if df['feattype'][k]['value']==ls:
                coords = np.array(df['geometry'][k].coords)
                for i in range(len(coords)-1):

                    c0,r0 = ~transform * tuple(coords[i])

                    c1,r1 = ~transform * tuple(coords[i+1])

                    rr, cc, val = line_aa(math.floor(r0),math.floor(c0),math.floor(r1), math.floor(c1))
                    new_val = np.ones(len(val))

                    whole_mask[rr, cc] = (new_val * color_mapping[ls])
    return(whole_mask)

In [98]:
#this function is used to append new polygon k after its transformation
#k est le 'Polygon' de clé k
#feature_nb est l'indice du feature dans la feature_list
def new_poly_append(k,feature_nb,df,poly_list):
    interior_coords = []
    #on teste si le 'Polygon' de clé k est du feature que l'on veut, si oui, on entre dans la boucle, si non, rien n'est ajouté à la poly_list
    if df['feattype'][k]['value']==features_list[feature_nb]:
        new_poly = df['geometry'][k]
        #we browse the features coming next in the list and modify the polygon if there is intersection
        #with polygons of those features
        for j,feature in enumerate(features_list[(feature_nb+1):]):
            df_polygon_geometry_feature = df[df["feattype"]== {'value': feature}].geometry
            for key2 in df_polygon_geometry_feature.keys():
                if new_poly.intersects(df_polygon_geometry_feature[key2])==True:
                    new_poly = new_poly.difference(df_polygon_geometry_feature[key2])
        poly_list.append(new_poly)
    return(poly_list)

def func_df(filepath):
    df = gpd.read_file(filepath)

    return(df)

def polygon_df(filepath,features_list):
    df = gpd.read_file(filepath)
    #ON NE GARDE QUE LES FEATURES QUI SONT DU TYPE 'Polygon'
    df=df[df["geometry"].geom_type == 'Polygon']
    
    #LISTE DES NOUVEAUX POLYGONES, ROGNÉS ET MODIFIÉS QUE L'ON AJOUTERA DANS LA COLONNE 'geometry2'
    poly_list=[]
    
    #PARCOURS DANS L'ORDRE DES 'Polygon'
    for k in df.id.keys():
        l = len(poly_list)
        for feature_nb in range(len(features_list)):
            poly_list = new_poly_append(k,feature_nb,df,poly_list)
        #CE TEST SERT À SAVOIR SI NEW_POLY_APPEND A AJOUTÉ UN ELEMENT, SINON C'EST QUE LE POLYGON DE CLÉ K N'EST PAS UN FEATURE INTERESSANT   
        if l == len(poly_list):
            poly_list.append(df['geometry'][k])
    df['geometry2']=poly_list
    df['geometry2']=gpd.GeoDataFrame(df['geometry2'])
    return(df)

In [99]:
# Fonctions de création de masques par lecture du GeoJson
# TO DO: V2 qui gère la superposition d'objets. 
# Idée: un petit objet inclut dans un autre doit avoir priorité sur l'affichage 

def poly_from_utm(polygon, transform):
    #liste les points exterieurs du polygon
    poly_pts = []
    
    #liste les 'trous' dans ce polygon sous forme de tuple
    poly_holes = []
   
    exterior_coords = polygon.exterior.coords[:]
    interior_coords = []
    for interior in polygon.interiors:
        interior_coords.append(tuple(interior.coords[:]))
    
    for i in np.array(exterior_coords):
        poly_pts.append(~transform * tuple(i))

    for hole in np.array(interior_coords):
        hole_pts=[]
        for k in hole:

            hole_pts.append(tuple(~transform * tuple(k)))
        poly_holes.append(tuple(hole_pts))
    new_poly = Polygon(poly_pts,poly_holes)
    return new_poly


# creating binary mask for field/not_filed segmentation.
def create_mask(features_list,src,df):
    im_size = (src.meta['height'], src.meta['width'])
    whole_mask = np.zeros([src.meta['height'],src.meta['width']])
                
    for i,f in enumerate(features_list):
        poly_shp = []
        data_geom = df[df['feattype'] == {'value': f}].geometry2
        k = 0
        for g in data_geom.values:

            if g.geom_type == 'Polygon':
                if not g.is_empty:

                    poly = poly_from_utm(g, src.meta['transform'])
                    poly_shp.append(poly)
                    poly_shp_polygon = poly_shp
            else:
                for p in g:
                    poly = poly_from_utm(p, src.meta['transform'])
                    poly_shp.append(poly)

        if poly_shp:
            mask = rasterize(shapes=poly_shp,out_shape=im_size)*color_mapping[f]
            whole_mask = whole_mask + mask
    return whole_mask


In [101]:
img_name = name_map+"_ortho.tif"
directory = os.fsencode(path_GeoTIFF)

poly_shp_test = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename==img_name:
        #print(path_GeoTIFF+"{}".format(filename))
        with rasterio.open(path_GeoTIFF+"{}".format(filename)) as src:
            img = src.read()
            meta = src.meta
            
            df = polygon_df(filepath,features_list)

            whole_mask= create_mask(features_list,src,df)
            
            whole_mask = draw_linestring(line_list,whole_mask,src.meta['transform'])
            
            # Just stores the different pixel values in the mask array, and print them.
            a=set([])
            
            for row in whole_mask:
                for elt in row:
                    a.add(elt)
            print("Pixel values: "+str(a))
            
            # Just prints warning if two objects are superimposed
            a.remove(0)
            for elt in a:
                boolean=False
                for key in color_mapping:
                    if(color_mapping[key] == elt):
                        boolean=True;
                if not boolean:
                    print("A superposition was detected !")
                    
            
            
            print(whole_mask.shape)
            
#             #Rescale to 0-255 and convert to uint8
#             rescaled = (255.0 / data.max() * (data - data.min())).astype(np.uint8)

            # Save numpy array into png mask 1 channel
            cv2.imwrite(outfolder + '/masktest_{}.png'.format(img_name.split('_')[0]),whole_mask)


Pixel values: {0.0, 160.0, 248.0, 100.0, 70.0, 200.0, 10.0, 240.0, 50.0, 180.0, 120.0, 220.0, 255.0}
(9299, 10400)


# END OF CODE

## Différents tests

In [ ]:
print(poly_shp_multi2poly)
#print(poly_shp_0)
im_size=(6200,7400)
mask = rasterize(shapes=poly_shp_multi2poly,out_shape=im_size)*color_mapping['Runway.runwayelement']
#print(mask)
#poly_shp[1]
cv2.imwrite(outfolder + '/masktesttotal.png',mask)
#poly_shp_polygon[1]

In [ ]:
main_list=[]
list_1=[(10.3880599184085, 43.6777942745563), (10.3877425683812, 43.6774866300529), (10.3877573619324, 43.6774786478598), (10.3880747119933, 43.6777862923223), (10.3880599184085, 43.6777942745563)]
main_list.append(tuple(list_1))
list_2=[(10.3880599184085, 43.6777942745563), (10.3877425683812, 43.6774866300529), (10.3877573619324, 43.6774786478598), (10.3880747119933, 43.6777862923223), (10.3880599184085, 43.6777942745563), (10.387513104236, 43.6772641811034), (10.3871944013245, 43.6769552181046), (10.3872091948176, 43.6769472359822), (10.3875278977629, 43.67725619894), (10.387513104236, 43.6772641811034)]
main_list.append(tuple(set(list_2) - set(list_1)))
print(main_list)

In [ ]:
inputPolygon = Polygon(((0,0),(10,0),(10,10),(0,10)), (((1,3),(5,3),(5,1),(1,1)), ((9,9),(9,8),(8,8),(8,9))))
polygonExterior = inputPolygon.exterior
polygonInteriors = []
for i in range(len(inputPolygon.interiors)):
    # do the stuff with your polygons
    polygonInteriors.append(inputPolygon.interiors[i])

newPolygon = Polygon(polygonExterior, [[pt for pt in inner.coords] for inner in polygonInteriors])
newPolygon

In [7]:
df=polygon_df(filepath,features_list)
df[df['feattype']=={'value': 'Taxiway.taxiwayintersectionmarking'}]

,id,feattype,label_id,name_id,aprontyp,docking,fuel,gndpower,idapron,jetway,...,wingspan,maxspeed,catstop,idlin,rwyahtxt,height,material,plysttyp,geometry,geometry2


# OLD: exploration et manipulation d'un GeoJson

### Open image with rasterio:

In [ ]:
with rasterio.open(path_GeoTIFF+"/"+name_map+"_ortho.tif") as src:
    img = src.read()
    meta = src.meta

img = reshape_as_image(img)



### Read Geojson as  a dataframe:

In [ ]:
def polygon_df(filepath):
    df = gpd.read_file(filepath)
    df = df.drop(columns=['originated', 'readonly',
           'notvalidated', 'lock', 'elev', 'hacc', 'iata', 'name', 'idarpt',
           'acft', 'idnumber', 'termref', 'pcn', 'restacft', 'status', 'surftype',
           'length', 'width', 'color', 'direc', 'style', 'rwymktyp', 'asda',
           'availPavedSurfFromThr', 'brngmag', 'brngtrue', 'cat', 'ellipse',
           'geound', 'lda', 'rops_landing_length', 'rwyslope', 'tdze', 'tdzslope',
           'thrtype', 'toda', 'tora', 'vasis', 'bridge', 'gsurftyp', 'runwayexit',
           'imagery_date'])
    
    return df[df["geometry"].geom_type == 'Polygon']

#### Only polygon:

In [ ]:
df = polygon_df(path_GeoJson+"product_ADBLucem_"+name_map+"1905_02.json")
df['geometry'].head()

### Liste des features contenues dans le GeoJSON

In [15]:

name_map='LFBO'
img_name = name_map+"_ortho.tif"
filepath = path_GeoJson#+"product_ADBLucem_"+name_map#+"_1912_0"+".json"

files=[]
for i in os.listdir(path_GeoJson):
    if os.path.isfile(os.path.join(path_GeoJson,i)) and name_map in i:
        files.append(i)

with rasterio.open(path_GeoTIFF+"{}".format(img_name)) as src:
    img = src.read()
    meta = src.meta
    #df = polygon_df(filepath,features_list)
    df = func_df(filepath+files[0])


features_list = []
for e in df['feattype']:
    if e['value'] not in features_list:
        features_list.append(e['value'])

print(features_list)

['Aerodrome.aerodromereferencepoint', 'Apron.apronelement', 'FinalApproachAndTakeOffArea.finalapproachandtakeoffarea', 'PaintedCenterline.paintedcenterline', 'ParkingStandArea.parkingstandarea', 'ParkingStandLocation.parkingstandlocation', 'Runway.runwayelement', 'Runway.runwayexitline', 'Runway.runwaymarking', 'Runway.runwayshoulder', 'Runway.runwaythreshold', 'ServiceRoad.serviceroad', 'StandGuidanceLine.standguidanceline', '0', 'Taxiway.taxiwayguidanceline', 'Taxiway.taxiwayholdingposition', 'Taxiway.taxiwayintersectionmarking', 'Taxiway.taxiwayshoulder', 'TouchDownLiftOffArea.touchdownliftoffarea', 'VerticalStructure.verticalpolygonalstructure', 'Water.water', 'ImageryData.imagerydata']


In [96]:
np.array(df[df['feattype']=={'value':'Taxiway.taxiwayguidanceline'}].geometry[1622].coords[1])

IndexError: index 1622 is out of bounds for axis 0 with size 0

### Présence de chaque feature dans le dataset

In [58]:
f='Runway.runwayelement'
compteur = 0
for i in os.listdir(path_GeoJson):
    if i.startswith('product'):
        df = func_df(filepath+i)

        deja_vu=False
        for e in df['feattype']:
            if e['value'] == f and not deja_vu:
                compteur+=1
                #comment following line if we want to know how many features are there in the total dataset
                #deja_vu=True

print('Dans notre dataset total, le feature '+str(f)+' apparait '+str(compteur)+' fois')

Dans notre dataset total, le feature Runway.runwayelement apparait 43 fois


In [72]:
df[df['feattype']=='Taxiway.taxiwayintersectionmarking']

,id,feattype,label_id,name_id,originated,readonly,notvalidated,lock,elev,hacc,...,runwayexit,maxspeed,catstop,idlin,rwyahtxt,height,material,plysttyp,imagery_date,geometry


In [ ]:
from shapely.geometry import Polygon
p1 = Polygon([(0,0), (1,1), (1,0)])
p2 = Polygon([(0,1), (1,0), (1,1)])
print(p1.intersects(p2))

In [ ]:
from shapely.geometry import Polygon

p = Polygon([(1,1),(2,2),(4,2),(3,1)])
q = Polygon([(1.5,2),(3,5),(5,4),(3.5,1)])
print(p.intersects(q))  # True
print(p.intersection(q).area)  # 1.0
x = p.intersection(q)
print(x)

In [ ]:
# multipol1 and multipol2 are my shapely MultiPolygons
from shapely.ops import cascaded_union
from itertools import combinations
from shapely.geometry import Polygon

print(data_geom[517])
outmulti = []
#for pol in data_geom:
#    for pol2 in data_geom2:
pol=data_geom[517]
pol2=pol
if pol.intersects(pol2)==True:
    print(True)
    # If they intersect, create a new polygon that is
    # essentially pol minus the intersection
    nonoverlap = (pol.symmetric_difference(pol2)).difference(pol2)
    outmulti.append(nonoverlap)
    
    print(nonoverlap.is_empty)
else:
    # Otherwise, just keep the initial polygon as it is.
    outmulti.append(pol)
print(data_geom[517])
#finalpol = MultiPolygon(outmulti)